In [1]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [4]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


GPU is configured properly.
Num GPUs Available: 1


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [38]:
f.predictions('amd')

AMD 5m Interval Timestamp: 2024-12-19 03:55:55 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.25521,0.262766,0.253012,0.18481
prob_up,0.249832,0.21025,0.200142,0.25
prob_static,0.522931,0.592332,0.592468,0.625
prob_down,0.227237,0.197418,0.20739,0.125
precision,0.468007,0.473404,0.466437,0.417722
recall,0.51927,0.541582,0.549696,0.535497
f1,0.492308,0.505203,0.504655,0.469333
support,"[495.0, 493.0, 493.0]","[495.0, 493.0, 493.0]","[495.0, 493.0, 493.0]","[495.0, 493.0, 493.0]"


AMD 15m Interval Timestamp: 2024-12-19 03:55:58 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,up
kelly_1:2.5,0.278351,0.034177,0.137107,0.036994
prob_up,0.405676,0.619471,0.472707,0.625
prob_static,0.409412,0.1547,0.269468,0.375
prob_down,0.184912,0.225829,0.257824,0.0
precision,0.484536,0.310127,0.383648,0.312139
recall,0.573171,0.29878,0.371951,0.329268
f1,0.52514,0.304348,0.377709,0.320475
support,"[164.0, 164.0, 164.0]","[164.0, 164.0, 164.0]","[164.0, 164.0, 164.0]","[164.0, 164.0, 164.0]"


AMD 1h Interval Timestamp: 2024-12-19 03:56:00 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,down,up
kelly_1:2.5,0.128715,0.126023,0.109986,0.06783
prob_up,0.296668,0.34745,0.324433,0.375
prob_static,0.352978,0.309092,0.327223,0.25
prob_down,0.350354,0.343457,0.348344,0.375
precision,0.377654,0.375731,0.364276,0.334165
recall,0.450067,0.343124,0.346257,0.35781
f1,0.410693,0.358688,0.355038,0.345583
support,"[749.0, 751.0, 748.0]","[749.0, 751.0, 748.0]","[749.0, 751.0, 748.0]","[749.0, 751.0, 748.0]"


AMD 1wk Interval Timestamp: 2024-12-19 03:56:08 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,up,up,up
kelly_1:2.5,0.140506,0.079452,0.18169,0.076389
prob_up,0.197954,0.42539,0.459753,0.375
prob_static,0.347377,0.215084,0.229292,0.25
prob_down,0.45467,0.359526,0.310955,0.375
precision,0.386076,0.342466,0.415493,0.340278
recall,0.393548,0.326797,0.385621,0.320261
f1,0.389776,0.334448,0.4,0.329966
support,"[153.0, 153.0, 155.0]","[153.0, 153.0, 155.0]","[153.0, 153.0, 155.0]","[153.0, 153.0, 155.0]"


AMD 1mo Interval Timestamp: 2024-12-19 03:56:09 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,static
kelly_1:2.5,0.144444,0.066667,0.180488,0.094118
prob_up,0.62486,0.399947,0.395265,0.375
prob_static,0.028872,0.303004,0.268193,0.5
prob_down,0.346268,0.297049,0.336542,0.125
precision,0.388889,0.333333,0.414634,0.352941
recall,0.451613,0.419355,0.548387,0.4
f1,0.41791,0.371429,0.472222,0.375
support,"[31.0, 30.0, 30.0]","[31.0, 30.0, 30.0]","[31.0, 30.0, 30.0]","[31.0, 30.0, 30.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')